# Import Libraries

In [ ]:
import os
import pandas as pd
import numpy as np
import sklearn
import librosa
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

# Adding Path to the Audio Folders

In [ ]:
data_dir = '/kaggle/input/audio-emotions/Emotions'
emotions = os.listdir(data_dir)

# Types of Audios

In [ ]:
print(emotions)

# List for storing X and y values

In [ ]:
features = []
labels = []

# Calculating the MCFF of each Audio and Storing label in y

In [ ]:
for emotion in emotions:
    # Get the path to the folder containing audio files for the current emotion
    emotion_dir = os.path.join(data_dir, emotion)
    
    for filename in os.listdir(emotion_dir):
        # Load the audio file using librosa
        audio_path = os.path.join(emotion_dir, filename)
        y, sr = librosa.load(audio_path, sr=None)
        
        # Extract features from the audio file (example: using MFCC, Zero Crossing Rate, Spectral Centroid, Spectral Rolloff)
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
         
        # Append the combined features and corresponding label to the lists
        features.append(mfcc)
        labels.append(emotion)

# Finding the Maximum in the 3rd Dimension

In [ ]:
a = 0
for feat in features:
    #print(feat.shape[1])
    if a < feat.shape[1]:
        a = feat.shape[1]
print(a)

In [ ]:
n = np.zeros((len(features),len(features[0]),a),dtype = float)

# Iterate through each feature and assign its values to the 'n' array
for i, feat in enumerate(features):
    n[i, :, :feat.shape[1]] = feat[ :, :feat.shape[1]]

# Changing Data to X and y

In [ ]:
X = n
y = np.array(labels)

# Splitting Data into Training and Testing

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Defining Kernel Value

In [ ]:
# Train an SVM classifier
svm = SVC(kernel ='poly', degree = 6 )

# Reshaping X from 3D to 2D, (because SVM takes 2D)

In [ ]:
n_samples = X_train.shape[0]
n_features = np.prod(X_train.shape[1:])  # Product of dimensions except for the first one
X_train_2d = X_train.reshape(n_samples, n_features)

In [ ]:

n_samples_test = X_test.shape[0]
n_features_test = np.prod(X_test.shape[1:])  # Product of dimensions except for the first one

# Ensure compatibility of reshaping
assert n_samples_test * n_features_test == X_test.flatten().size, "Sizes are not compatible for reshaping"

X_test_2d = X_test.reshape(n_samples_test, n_features_test)


# Training Model with Data

In [ ]:
# Now, you can fit the SVM classifier
svm.fit(X_train_2d, y_train)

# Taking Predictions

In [ ]:
# Make predictions on the testing set
y_pred = svm.predict(X_test_2d)

# Checking Accuracy

In [ ]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Classification Report

In [ ]:
classification_report(y_test,y_pred)